# Shark League Score
## Import Python libraries



In [3]:
import pandas as pd
import numpy as np
import logging
import swc_simple_client as swc

In [4]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='shark_notebook.log',  
    level=logging.INFO,  
)

## Setup notebook variables

In [6]:
base_url = "https://humble-potato-r4grv6qx9vw9f9qj-8000.app.github.dev/"


## Get Custom Max Scores
### Use endpoint: LIST_WEEKS_ENDPOINT 

In [9]:
week_api_response = swc.call_api_endpoint(base_url,swc.LIST_WEEKS_ENDPOINT)
weeks_df = pd.DataFrame(week_api_response.json())
weeks_df['year'] = weeks_df['week_number'].str.slice(0, 4).astype(int)
weeks_df['week'] = weeks_df['week_number'].str.slice(4, 6).astype(int)



max_totals_grouped_df = weeks_df.groupby('year').agg(
    ppr_12_max_points=('ppr_12_max_points', 'sum'), 
    half_ppr_8_max_points=('half_ppr_8_max_points', 'sum'))

display(max_totals_grouped_df)

,ppr_12_max_points,half_ppr_8_max_points
year,,
2023,22570.0,15401.0


## Get League Scoring Type
### Use Endpoint: LIST_LEAGUES_ENDPOINT

In [10]:
league_api_response = swc.call_api_endpoint(base_url,swc.LIST_LEAGUES_ENDPOINT)
leagues_df = pd.DataFrame(league_api_response.json())
leagues_df = leagues_df.drop(columns=['teams','last_changed_date'])
display(leagues_df)

,league_id,league_name,scoring_type,league_size
0,5001,Pigskin Prodigal Fantasy League,PPR,12
1,5002,Recurring Champions League,Half-PPR,8
2,5003,AHAHFZZFFFL,Half-PPR,8
3,5004,Gridiron Gurus Fantasy League,PPR,12
4,5005,Best League Ever,PPR,12


## Get Regular Season Scoring Totals - By Team
### Use Endpoint: LIST_TEAMS_ENDPOINT

In [12]:
team_api_response = swc.call_api_endpoint(base_url,swc.LIST_TEAMS_ENDPOINT)

teams_parsed_df = pd.json_normalize(team_api_response.json(), 'weekly_scores', 
                                    ['league_id', 'team_id', 'team_name'])

teams_parsed_df['year'] = (teams_parsed_df['week_number']
                           .str.slice(0, 4).astype(int))
teams_parsed_df['week'] = (teams_parsed_df['week_number']
                           .str.slice(4, 6).astype(int))

#get only regular season teams
teams_regular_season_df = teams_parsed_df.query('week <= 14')

#get team season totals
team_totals_df = teams_regular_season_df.groupby(
    ['league_id', 'team_id', 'year'], as_index = False
    )['fantasy_points'].sum()

team_totals_df.head()

,league_id,team_id,year,fantasy_points
0,5001,1001,2023,1238.0
1,5001,1002,2023,1216.0
2,5001,1003,2023,1193.0
3,5001,1004,2023,1071.0
4,5001,1005,2023,958.0


In [ ]:
“echo -e "AIRFLOW_UID=$(id -u)" > .env

”

Excerpt From
Hands-On APIs for AI and Data Science
Ryan Day
This material may be protected by copyright.